# **Food Data SQL**

# **Libraries Used:**

In [1]:
import pandas as pd
import sqlite3

# **Connect DataBase**

In [2]:
conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

# **Create Tables:**

Providers Table:

In [3]:
cursor.execute("""
CREATE TABLE providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
);
""")

Receivers Table:

In [4]:
cursor.execute("""
CREATE TABLE receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT,
    City TEXT,
    Contact TEXT
);
""")

Listings Table:

In [5]:
cursor.execute("""
CREATE TABLE food_listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT NOT NULL,
    Quantity INTEGER,
    Expiry_Date TEXT,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID) ON DELETE CASCADE
);
""")

Claims Table:

In [6]:
cursor.execute("""
CREATE TABLE claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp TEXT,
    FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID) ON DELETE CASCADE,
    FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID) ON DELETE CASCADE
);
""")

Commit:

In [7]:
conn.commit()

# **Insert Data:**

Load Data:

In [8]:
providers_df = pd.read_csv("data/providers_data.csv")
receivers_df = pd.read_csv("data/receivers_data.csv")
food_df = pd.read_csv("data/food_listings_data.csv")
claims_df = pd.read_csv("data/claims_data.csv")

Insert Data into Tables:

In [9]:
providers_df.to_sql("providers", conn, if_exists="append", index=False)
receivers_df.to_sql("receivers", conn, if_exists="append", index=False)
food_df.to_sql("food_listings", conn, if_exists="append", index=False)
claims_df.to_sql("claims", conn, if_exists="append", index=False)

1000

Commit:

In [10]:
conn.commit()

# **Queries:**

1. How many food providers and receivers are there in each city?

In [ ]:
query1 = """
SELECT p.City,
       COUNT(DISTINCT p.Provider_ID) AS Total_Providers,
       COUNT(DISTINCT r.Receiver_ID) AS Total_Receivers
FROM providers p
LEFT JOIN receivers r
ON p.City = r.City
GROUP BY p.City;
"""
df1 = pd.read_sql_query(query1, conn)
df1

,City,Total_Providers,Total_Receivers
0,Adambury,1,0
1,Adamsview,1,0
2,Adamsville,1,0
3,Aguirreville,1,0
4,Alexanderchester,1,0
...,...,...,...
958,Wrightville,1,0
959,Yatesside,1,0
960,Youngchester,1,0
961,Zimmermanton,1,0


2. Which type of food provider contributes the most food?

In [19]:
query2 = """
SELECT Provider_Type,
       SUM(Quantity) AS Total_Quantity
FROM food_listings
GROUP BY Provider_Type
ORDER BY Total_Quantity DESC
LIMIT 1;
"""
df2 = pd.read_sql_query(query2, conn)
df2

,Provider_Type,Total_Quantity
0,Restaurant,6923


3. Contact information of food providers in a specific city (Example: Mumbai)

In [ ]:
query3 = """
SELECT Name, Type, Contact
FROM providers
WHERE City = 'Kellytown';
"""
df3 = pd.read_sql_query(query3, conn)
df3

,Name,Type,Contact
0,Cannon-Garcia,Restaurant,9421508200


4. Which receivers have claimed the most food?

In [24]:
query4 = """
SELECT r.Name,
       COUNT(c.Claim_ID) AS Total_Claims
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Name
ORDER BY Total_Claims DESC
LIMIT 5;
"""
df4 = pd.read_sql_query(query4, conn)
df4

,Name,Total_Claims
0,William Frederick,5
1,Scott Hunter,5
2,Matthew Webb,5
3,Anthony Garcia,5
4,Kristine Martin,4


5. Total quantity of food available from all providers

In [26]:
query5 = """
SELECT SUM(Quantity) AS Total_Foods
FROM food_listings;
"""
df5 = pd.read_sql_query(query5, conn)
df5

,Total_Foods
0,25794


6. Which city has the highest number of food listings?

In [27]:
query6 = """
SELECT Location,
       COUNT(Food_ID) AS Total_Listings
FROM food_listings
GROUP BY Location
ORDER BY Total_Listings DESC
LIMIT 1;
"""
df6 = pd.read_sql_query(query6, conn)
df6

,Location,Total_Listings
0,South Kathryn,6


7. Most commonly available food types

In [28]:
query7 = """
SELECT Food_Type,
       COUNT(*) AS Count_Type
FROM food_listings
GROUP BY Food_Type
ORDER BY Count_Type DESC;
"""
df7 = pd.read_sql_query(query7, conn)
df7

,Food_Type,Count_Type
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


8. Number of food claims for each food item

In [29]:
query8 = """
SELECT f.Food_Name,
       COUNT(c.Claim_ID) AS Total_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Food_Name
ORDER BY Total_Claims DESC;
"""
df8 = pd.read_sql_query(query8, conn)
df8

,Food_Name,Total_Claims
0,Rice,122
1,Soup,114
2,Dairy,110
3,Fish,108
4,Salad,106
5,Chicken,102
6,Bread,94
7,Pasta,87
8,Vegetables,86
9,Fruits,71


9. Provider with the highest number of successful claims

In [32]:
query9 = """
SELECT p.Name,
       COUNT(c.Claim_ID) AS Successful_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN providers p ON f.Provider_ID = p.Provider_ID
WHERE c.Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC
LIMIT 1;
"""
df9 = pd.read_sql_query(query9, conn)
df9

,Name,Successful_Claims
0,Barry Group,5


10. Percentage of claims by status

In [35]:
query10 = """
SELECT Status,
       ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims), 2) AS Percentage
FROM claims
GROUP BY Status;
"""
df10 = pd.read_sql_query(query10, conn)
df10

,Status,Percentage
0,Cancelled,33.6
1,Completed,33.9
2,Pending,32.5


11. Average quantity of food claimed per receiver

In [36]:
query11 = """
SELECT r.Name,
       ROUND(AVG(f.Quantity), 2) AS Avg_Quantity_Claimed
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Name;
"""
df11 = pd.read_sql_query(query11, conn)
df11

,Name,Avg_Quantity_Claimed
0,Aaron Keller,39.00
1,Aaron Rios,21.00
2,Aaron Scott,45.00
3,Abigail Crawford,25.67
4,Adam Browning,5.00
...,...,...
615,William Barnes,47.00
616,William Fox,12.00
617,William Frederick,21.40
618,Yvette Huffman,45.00


12. Most claimed meal type

In [37]:
query12 = """
SELECT Meal_Type,
       COUNT(*) AS Claim_Count
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY Meal_Type
ORDER BY Claim_Count DESC
LIMIT 1;
"""
df12 = pd.read_sql_query(query12, conn)
df12

,Meal_Type,Claim_Count
0,Breakfast,278


13. Total quantity of food donated by each provider

In [38]:
query13 = """
SELECT p.Name,
       SUM(f.Quantity) AS Total_Donated
FROM food_listings f
JOIN providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated DESC;
"""
df13 = pd.read_sql_query(query13, conn)
df13

,Name,Total_Donated
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145
...,...,...
623,Reyes and Sons,1
624,Norris-Townsend,1
625,Mcgee PLC,1
626,"Martinez, Armstrong and Carroll",1


14. Top 5 cities with most completed claims

In [39]:
query14 = """
SELECT f.Location,
       COUNT(c.Claim_ID) AS Completed_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY f.Location
ORDER BY Completed_Claims DESC
LIMIT 5;
"""
df14 = pd.read_sql_query(query14, conn)
df14

,Location,Completed_Claims
0,South Kathryn,5
1,Zimmermanville,4
2,New Carol,4
3,East Heatherport,4
4,Devinmouth,4


15. Food items nearing expiry (next 3 days)

In [40]:
query15 = """
SELECT Food_Name, Expiry_Date, Quantity
FROM food_listings
WHERE date(Expiry_Date) <= date('now', '+3 days');
"""
df15 = pd.read_sql_query(query15, conn)
df15

,Food_Name,Expiry_Date,Quantity


Close:

In [ ]:
conn.close()